In [521]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer

import warnings; warnings.simplefilter('ignore')


In [522]:
md = pd.read_csv('../Datasets/movies_dataset.csv', lineterminator='\n')
md.head(n = 40)

,genre_ids,id,original_language,overview,popularity,release_date,title,vote_average,vote_count
0,"[28, 14, 878]",436270,en,"Nearly 5,000 years after he was bestowed with ...",11801.698,2022-10-19,Black Adam,7.3,2197
1,"[14, 28, 35, 80]",1013860,en,When Sheriff Roy Pulsipher finds himself in th...,4442.620,2022-11-15,R.I.P.D. 2: Rise of the Damned,6.8,175
2,"[80, 28, 53]",829799,en,Renegade bounty hunter Ryan Swan must carve hi...,3475.823,2022-11-11,Paradise City,6.3,28
3,"[878, 28]",872177,en,"Set in San Tiburon, the world's most dangerous...",3197.165,2022-04-29,Corrective Measures,5.0,31
4,"[28, 27, 53]",988233,en,Following a mysterious disappearance on a jump...,3648.142,2022-11-01,Hex,4.3,6
5,"[28, 18, 36]",724495,en,"The story of the Agojie, the all-female unit o...",4376.054,2022-09-15,The Woman King,7.9,526
6,"[80, 18, 9648, 53]",862965,en,"Emily, who is saddled with student debt and lo...",2369.199,2022-08-12,Emily the Criminal,6.9,211
7,"[53, 80]",934641,en,A stockbroker in a small southern town gets em...,2358.978,2022-11-04,The Minute You Wake Up Dead,4.9,19
8,"[35, 10751, 10402]",830784,en,"When the Primm family moves to New York City, ...",1936.367,2022-10-07,"Lyle, Lyle, Crocodile",7.7,91
9,"[35, 10751, 14]",338958,en,"Disillusioned with life in the city, feeling o...",1969.641,2022-11-16,Disenchanted,7.3,428


In [523]:
kywrds = pd.read_csv('../Datasets/keywords_dataframe.csv')
md = md.merge(kywrds, on='id')

In [524]:
md.head()
# print(md.iloc[0])
# print(md.iloc[0]['keywords'])

,genre_ids,id,original_language,overview,popularity,release_date,title,vote_average,vote_count,Unnamed: 0,keywords
0,"[28, 14, 878]",436270,en,"Nearly 5,000 years after he was bestowed with ...",11801.698,2022-10-19,Black Adam,7.3,2197,87,"[{'id': 952, 'name': 'lightning'}, {'id': 2095..."
1,"[14, 28, 35, 80]",1013860,en,When Sheriff Roy Pulsipher finds himself in th...,4442.620,2022-11-15,R.I.P.D. 2: Rise of the Damned,6.8,175,14,"[{'id': 798, 'name': 'sheriff'}, {'id': 9717, ..."
2,"[80, 28, 53]",829799,en,Renegade bounty hunter Ryan Swan must carve hi...,3475.823,2022-11-11,Paradise City,6.3,28,56,[]
3,"[878, 28]",872177,en,"Set in San Tiburon, the world's most dangerous...",3197.165,2022-04-29,Corrective Measures,5.0,31,6,"[{'id': 9717, 'name': 'based on comic'}, {'id'..."
4,"[28, 27, 53]",988233,en,Following a mysterious disappearance on a jump...,3648.142,2022-11-01,Hex,4.3,6,18,"[{'id': 4679, 'name': 'parachuting'}, {'id': 9..."


In [525]:
md = md.drop(labels=['genre_ids', 'Unnamed: 0'], axis=1)

In [526]:
md.head()

,id,original_language,overview,popularity,release_date,title,vote_average,vote_count,keywords
0,436270,en,"Nearly 5,000 years after he was bestowed with ...",11801.698,2022-10-19,Black Adam,7.3,2197,"[{'id': 952, 'name': 'lightning'}, {'id': 2095..."
1,1013860,en,When Sheriff Roy Pulsipher finds himself in th...,4442.620,2022-11-15,R.I.P.D. 2: Rise of the Damned,6.8,175,"[{'id': 798, 'name': 'sheriff'}, {'id': 9717, ..."
2,829799,en,Renegade bounty hunter Ryan Swan must carve hi...,3475.823,2022-11-11,Paradise City,6.3,28,[]
3,872177,en,"Set in San Tiburon, the world's most dangerous...",3197.165,2022-04-29,Corrective Measures,5.0,31,"[{'id': 9717, 'name': 'based on comic'}, {'id'..."
4,988233,en,Following a mysterious disappearance on a jump...,3648.142,2022-11-01,Hex,4.3,6,"[{'id': 4679, 'name': 'parachuting'}, {'id': 9..."


In [527]:
import ast

In [528]:
def convert(obj):
    lst = []
    for dcts in ast.literal_eval(obj):
        lst.append(dcts['name'])
    return lst

In [529]:
md.iloc[0]['keywords']

"[{'id': 952, 'name': 'lightning'}, {'id': 2095, 'name': 'anti hero'}, {'id': 9715, 'name': 'superhero'}, {'id': 9717, 'name': 'based on comic'}, {'id': 15001, 'name': 'demon'}, {'id': 155030, 'name': 'superhero team'}, {'id': 179431, 'name': 'duringcreditsstinger'}, {'id': 229266, 'name': 'dc extended universe'}]"

In [530]:
# print(md.iloc[0]['keywords'])
# print(type(md.iloc[0]['keywords']))
# lst = ast.literal_eval(md.iloc[0]['keywords'])
# lst = convert(lst)
# print(lst)

In [531]:
md['keywords'] = md['keywords'].apply(convert)

In [532]:
md.head()

,id,original_language,overview,popularity,release_date,title,vote_average,vote_count,keywords
0,436270,en,"Nearly 5,000 years after he was bestowed with ...",11801.698,2022-10-19,Black Adam,7.3,2197,"[lightning, anti hero, superhero, based on com..."
1,1013860,en,When Sheriff Roy Pulsipher finds himself in th...,4442.620,2022-11-15,R.I.P.D. 2: Rise of the Damned,6.8,175,"[sheriff, based on comic, american west, 1870s..."
2,829799,en,Renegade bounty hunter Ryan Swan must carve hi...,3475.823,2022-11-11,Paradise City,6.3,28,[]
3,872177,en,"Set in San Tiburon, the world's most dangerous...",3197.165,2022-04-29,Corrective Measures,5.0,31,"[based on comic, lockdown, prison measures]"
4,988233,en,Following a mysterious disappearance on a jump...,3648.142,2022-11-01,Hex,4.3,6,"[parachuting, paranormal, skydiving]"


In [533]:
md = md.rename(columns={'keywords': 'genres'})

In [534]:
md.head()

,id,original_language,overview,popularity,release_date,title,vote_average,vote_count,genres
0,436270,en,"Nearly 5,000 years after he was bestowed with ...",11801.698,2022-10-19,Black Adam,7.3,2197,"[lightning, anti hero, superhero, based on com..."
1,1013860,en,When Sheriff Roy Pulsipher finds himself in th...,4442.620,2022-11-15,R.I.P.D. 2: Rise of the Damned,6.8,175,"[sheriff, based on comic, american west, 1870s..."
2,829799,en,Renegade bounty hunter Ryan Swan must carve hi...,3475.823,2022-11-11,Paradise City,6.3,28,[]
3,872177,en,"Set in San Tiburon, the world's most dangerous...",3197.165,2022-04-29,Corrective Measures,5.0,31,"[based on comic, lockdown, prison measures]"
4,988233,en,Following a mysterious disappearance on a jump...,3648.142,2022-11-01,Hex,4.3,6,"[parachuting, paranormal, skydiving]"


In [535]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_average = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = round(vote_average.mean())
C

6

In [536]:
m = vote_counts.quantile(0.85)
m

2582.5999999999985

In [537]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [538]:
md.head()

,id,original_language,overview,popularity,release_date,title,vote_average,vote_count,genres,year
0,436270,en,"Nearly 5,000 years after he was bestowed with ...",11801.698,2022-10-19,Black Adam,7.3,2197,"[lightning, anti hero, superhero, based on com...",2022
1,1013860,en,When Sheriff Roy Pulsipher finds himself in th...,4442.620,2022-11-15,R.I.P.D. 2: Rise of the Damned,6.8,175,"[sheriff, based on comic, american west, 1870s...",2022
2,829799,en,Renegade bounty hunter Ryan Swan must carve hi...,3475.823,2022-11-11,Paradise City,6.3,28,[],2022
3,872177,en,"Set in San Tiburon, the world's most dangerous...",3197.165,2022-04-29,Corrective Measures,5.0,31,"[based on comic, lockdown, prison measures]",2022
4,988233,en,Following a mysterious disappearance on a jump...,3648.142,2022-11-01,Hex,4.3,6,"[parachuting, paranormal, skydiving]",2022


In [539]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['id', 'title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]

In [540]:
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(1500, 7)

In [541]:
def weighted_rating(obj):
    v = obj['vote_count']
    R = obj['vote_average']
    return (v/(v+m)*R) + (m/(v+m)*C)

In [542]:
qualified['wr'] = qualified.apply(weighted_rating, axis = 1)

In [543]:
qualified.head()

,id,title,year,vote_count,vote_average,popularity,genres,wr
37,616037,Thor: Love and Thunder,2022,4790,6,516.146,"[ex-girlfriend, hero, greek mythology, sequel,...",6.000000
40,507086,Jurassic World Dominion,2022,3917,7,485.621,"[giant monster, dinosaur, child kidnapping, ju...",6.602652
44,634649,Spider-Man: No Way Home,2021,15948,8,480.570,"[new york city, loss of loved one, showdown, s...",7.721261
48,19995,Avatar,2009,26644,7,498.460,"[culture clash, future, space war, space colon...",6.911635
49,361743,Top Gun: Maverick,2022,4871,8,466.170,"[fighter pilot, u.s. navy, sequel, nuclear wea...",7.307019


In [544]:
md.iloc[112]

id                                                              299536
original_language                                                   en
overview             As the Avengers and their allies have continue...
popularity                                                     195.412
release_date                                                2018-04-25
title                                           Avengers: Infinity War
vote_average                                                       8.3
vote_count                                                       25795
genres               [magic, sacrifice, superhero, based on comic, ...
year                                                              2018
Name: 112, dtype: object

In [545]:
qualified = qualified.sort_values(by = 'wr', ascending=False)

In [546]:
qualified.head()

,id,title,year,vote_count,vote_average,popularity,genres,wr
491,27205,Inception,2010,32671,8,71.793,"[paris, france, spy, philosophy, allegory, dre...",7.853484
233,157336,Interstellar,2014,29874,8,103.146,"[artificial intelligence, nasa, spacecraft, ex...",7.840858
547,155,The Dark Knight,2008,28646,8,66.709,"[crime fighter, secret identity, anti hero, sc...",7.834600
112,299536,Avengers: Infinity War,2018,25795,8,195.412,"[magic, sacrifice, superhero, based on comic, ...",7.817983
286,550,Fight Club,1999,25238,8,91.060,"[based on novel or book, support group, dual i...",7.814339


In [547]:
# import pickle

In [548]:
# pickle.dump(md, open('allMovies.pkl', 'wb'))

In [549]:
# pickle.dump(qualified, open('topWorldMovies.pkl', 'wb'))

In [550]:
qualified['title'].values

array(['Inception', 'Interstellar', 'The Dark Knight', ..., 'Death Note',
       'Batman & Robin', 'Fantastic Four'], dtype=object)

In [551]:
qualified.iloc[0]['id'].astype('int')

27205

### The above was just a simple recommendation system

In [552]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [553]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    print(df.head(n = 2))
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['id', 'title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [554]:
build_chart('sports')

         id original_language  \
280  379686                en   
307  348893                en   

                                              overview  popularity  \
280  When LeBron and his young son Dom are trapped ...     102.111   
307  In the fourth installment of the fighting fran...      86.887   

    release_date                    title  vote_average  vote_count  year  \
280   2021-07-08  Space Jam: A New Legacy           6.9        3352  2021   
307   2016-08-01     Boyka: Undisputed IV           7.0        1067  2016   

      genre  
280  sports  
307  sports  


,id,title,year,vote_count,vote_average,popularity,wr
987,359724,Ford v Ferrari,2019,6167,8,46.162,7.537495
3300,1578,Raging Bull,1980,3529,8,22.260,7.325591
921,1366,Rocky,1976,6433,7,38.776,6.809007
452,312221,Creed,2015,6227,7,68.746,6.804354
2403,96721,Rush,2013,6158,7,22.096,6.802745
424,22881,The Blind Side,2009,5375,7,78.705,6.782432
3117,389015,"I, Tonya",2017,4997,7,18.777,6.771051
715,307081,Southpaw,2015,4868,7,56.520,6.766889
375,480530,Creed II,2018,4417,7,68.923,6.751069
3477,60308,Moneyball,2011,4232,7,19.127,6.743942


In [555]:
# based on novel or book
# murder
# romance
# superhero
# sports
top_genres = ['based on novel or book', 'murder', 'romance', 'superhero', 'sports', 'biography', 'friendship', 'based on comic', ]

# for genre in top_genres:
#     chart = build_chart(genre = genre)
#     name = genre + 'Movies.pkl'
#     pickle.dump(chart, open(name, 'wb'))

### All above was simple recommender and just some genre classifications:

#### Now is the main CONTENT based recommender

In [556]:
md

,id,original_language,overview,popularity,release_date,title,vote_average,vote_count,genres,year
0,436270,en,"Nearly 5,000 years after he was bestowed with ...",11801.698,2022-10-19,Black Adam,7.3,2197,"[lightning, anti hero, superhero, based on com...",2022
1,1013860,en,When Sheriff Roy Pulsipher finds himself in th...,4442.620,2022-11-15,R.I.P.D. 2: Rise of the Damned,6.8,175,"[sheriff, based on comic, american west, 1870s...",2022
2,829799,en,Renegade bounty hunter Ryan Swan must carve hi...,3475.823,2022-11-11,Paradise City,6.3,28,[],2022
3,872177,en,"Set in San Tiburon, the world's most dangerous...",3197.165,2022-04-29,Corrective Measures,5.0,31,"[based on comic, lockdown, prison measures]",2022
4,988233,en,Following a mysterious disappearance on a jump...,3648.142,2022-11-01,Hex,4.3,6,"[parachuting, paranormal, skydiving]",2022
...,...,...,...,...,...,...,...,...,...,...
9995,801058,en,"Passionate about ocean life, a filmmaker sets ...",8.497,2021-03-24,Seaspiracy,7.9,328,"[fishing, pollution, ecology, environmental de...",2021
9996,152581,en,Animated film based on the wonderful children'...,10.381,2012-12-25,Room on the Broom,6.4,72,"[witch, cat, dragon, based on children's book,...",2012
9997,12506,en,"A dramatic story, based on actual events, abou...",8.467,1987-11-06,Cry Freedom,7.1,162,"[journalist, 1970s, male friendship, south afr...",1987
9998,1613,en,An American master chemist plans to score big ...,12.968,2001-12-07,The 51st State,6.1,488,"[chemical, gangster, chase, northern england, ...",2001


In [557]:
cast_df = pd.read_csv('../Datasets/cast_dataset.csv')
crew_df = pd.read_csv('../Datasets/crew_dataset.csv')
cast_df.head()

,id,cast
0,979924,"[{'cast_id': 2461, 'name': 'Mel Gibson', 'char..."
1,632856,"[{'cast_id': 23659, 'name': 'Will Ferrell', 'c..."
2,1013860,"[{'cast_id': 52886, 'name': 'Jeffrey Donovan',..."
3,551271,"[{'cast_id': 11107, 'name': 'Ben Foster', 'cha..."
4,505642,"[{'cast_id': 1083010, 'name': 'Letitia Wright'..."


In [558]:
crew_df

,id,crew
0,979924,"[{'crew_id': 94451, 'name': 'Robert Ogden Barn..."
1,632856,"[{'crew_id': 3557, 'name': 'Charles Dickens', ..."
2,1013860,"[{'crew_id': 22141, 'name': 'Ogden Gavanski', ..."
3,551271,"[{'crew_id': 2484, 'name': 'Steven Rosenblum',..."
4,505642,"[{'crew_id': 7232, 'name': 'Sarah Halley Finn'..."
...,...,...
9995,14583,"[{'crew_id': 74429, 'name': 'Josh Pate', 'depa..."
9996,25431,"[{'crew_id': 3633, 'name': 'Lawrence Weingarte..."
9997,18191,"[{'crew_id': 1255, 'name': 'David Mamet', 'dep..."
9998,8046,"[{'crew_id': 3117, 'name': 'Gary Frutkoff', 'd..."


In [559]:
md = md.merge(cast_df, on = 'id')
md = md.merge(crew_df, on = 'id')

In [560]:
md

,id,original_language,overview,popularity,release_date,title,vote_average,vote_count,genres,year,cast,crew
0,436270,en,"Nearly 5,000 years after he was bestowed with ...",11801.698,2022-10-19,Black Adam,7.3,2197,"[lightning, anti hero, superhero, based on com...",2022,"[{'cast_id': 18918, 'name': 'Dwayne Johnson', ...","[{'crew_id': 3687, 'name': 'Harry Cohen', 'dep..."
1,1013860,en,When Sheriff Roy Pulsipher finds himself in th...,4442.620,2022-11-15,R.I.P.D. 2: Rise of the Damned,6.8,175,"[sheriff, based on comic, american west, 1870s...",2022,"[{'cast_id': 52886, 'name': 'Jeffrey Donovan',...","[{'crew_id': 22141, 'name': 'Ogden Gavanski', ..."
2,829799,en,Renegade bounty hunter Ryan Swan must carve hi...,3475.823,2022-11-11,Paradise City,6.3,28,[],2022,"[{'cast_id': 8891, 'name': 'John Travolta', 'c...","[{'crew_id': 26713, 'name': 'Chuck Russell', '..."
3,872177,en,"Set in San Tiburon, the world's most dangerous...",3197.165,2022-04-29,Corrective Measures,5.0,31,"[based on comic, lockdown, prison measures]",2022,"[{'cast_id': 62, 'name': 'Bruce Willis', 'char...","[{'crew_id': 131416, 'name': ""Sean Patrick O'R..."
4,988233,en,Following a mysterious disappearance on a jump...,3648.142,2022-11-01,Hex,4.3,6,"[parachuting, paranormal, skydiving]",2022,"[{'cast_id': 1518164, 'name': 'Kayla Adams', '...","[{'crew_id': 55873, 'name': 'Hans Rodionoff', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,801058,en,"Passionate about ocean life, a filmmaker sets ...",8.497,2021-03-24,Seaspiracy,7.9,328,"[fishing, pollution, ecology, environmental de...",2021,"[{'cast_id': 3024859, 'name': 'Ali Tabrizi', '...","[{'crew_id': 1676969, 'name': 'Kip Andersen', ..."
9996,152581,en,Animated film based on the wonderful children'...,10.381,2012-12-25,Room on the Broom,6.4,72,"[witch, cat, dragon, based on children's book,...",2012,"[{'cast_id': 12214, 'name': 'Gillian Anderson'...","[{'crew_id': 592851, 'name': 'Julia Donaldson'..."
9997,12506,en,"A dramatic story, based on actual events, abou...",8.467,1987-11-06,Cry Freedom,7.1,162,"[journalist, 1970s, male friendship, south afr...",1987,"[{'cast_id': 8945, 'name': 'Kevin Kline', 'cha...","[{'crew_id': 1302, 'name': 'Susie Figgis', 'de..."
9998,1613,en,An American master chemist plans to score big ...,12.968,2001-12-07,The 51st State,6.1,488,"[chemical, gangster, chase, northern england, ...",2001,"[{'cast_id': 2231, 'name': 'Samuel L. Jackson'...","[{'crew_id': 2231, 'name': 'Samuel L. Jackson'..."


In [561]:
def convert3(obj):
    lst = []
    counter = 0
    for dcts in ast.literal_eval(obj):
        lst.append(dcts['name'])
        counter += 1
        if counter == 3:
            break
    return lst

In [562]:
md['cast'] = md['cast'].apply(convert3)

In [563]:
md

,id,original_language,overview,popularity,release_date,title,vote_average,vote_count,genres,year,cast,crew
0,436270,en,"Nearly 5,000 years after he was bestowed with ...",11801.698,2022-10-19,Black Adam,7.3,2197,"[lightning, anti hero, superhero, based on com...",2022,"[Dwayne Johnson, Aldis Hodge, Noah Centineo]","[{'crew_id': 3687, 'name': 'Harry Cohen', 'dep..."
1,1013860,en,When Sheriff Roy Pulsipher finds himself in th...,4442.620,2022-11-15,R.I.P.D. 2: Rise of the Damned,6.8,175,"[sheriff, based on comic, american west, 1870s...",2022,"[Jeffrey Donovan, Penelope Mitchell, Richard B...","[{'crew_id': 22141, 'name': 'Ogden Gavanski', ..."
2,829799,en,Renegade bounty hunter Ryan Swan must carve hi...,3475.823,2022-11-11,Paradise City,6.3,28,[],2022,"[John Travolta, Bruce Willis, Blake Jenner]","[{'crew_id': 26713, 'name': 'Chuck Russell', '..."
3,872177,en,"Set in San Tiburon, the world's most dangerous...",3197.165,2022-04-29,Corrective Measures,5.0,31,"[based on comic, lockdown, prison measures]",2022,"[Bruce Willis, Hayley Sales, Michael Rooker]","[{'crew_id': 131416, 'name': ""Sean Patrick O'R..."
4,988233,en,Following a mysterious disappearance on a jump...,3648.142,2022-11-01,Hex,4.3,6,"[parachuting, paranormal, skydiving]",2022,"[Kayla Adams, Matthew Holcomb, Bryan David Rob...","[{'crew_id': 55873, 'name': 'Hans Rodionoff', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,801058,en,"Passionate about ocean life, a filmmaker sets ...",8.497,2021-03-24,Seaspiracy,7.9,328,"[fishing, pollution, ecology, environmental de...",2021,"[Ali Tabrizi, Sylvia Earle, Richard O'Barry]","[{'crew_id': 1676969, 'name': 'Kip Andersen', ..."
9996,152581,en,Animated film based on the wonderful children'...,10.381,2012-12-25,Room on the Broom,6.4,72,"[witch, cat, dragon, based on children's book,...",2012,"[Gillian Anderson, Timothy Spall, Sally Hawkins]","[{'crew_id': 592851, 'name': 'Julia Donaldson'..."
9997,12506,en,"A dramatic story, based on actual events, abou...",8.467,1987-11-06,Cry Freedom,7.1,162,"[journalist, 1970s, male friendship, south afr...",1987,"[Kevin Kline, Denzel Washington, Penelope Wilton]","[{'crew_id': 1302, 'name': 'Susie Figgis', 'de..."
9998,1613,en,An American master chemist plans to score big ...,12.968,2001-12-07,The 51st State,6.1,488,"[chemical, gangster, chase, northern england, ...",2001,"[Samuel L. Jackson, Robert Carlyle, Emily Mort...","[{'crew_id': 2231, 'name': 'Samuel L. Jackson'..."


In [564]:
md.head()

,id,original_language,overview,popularity,release_date,title,vote_average,vote_count,genres,year,cast,crew
0,436270,en,"Nearly 5,000 years after he was bestowed with ...",11801.698,2022-10-19,Black Adam,7.3,2197,"[lightning, anti hero, superhero, based on com...",2022,"[Dwayne Johnson, Aldis Hodge, Noah Centineo]","[{'crew_id': 3687, 'name': 'Harry Cohen', 'dep..."
1,1013860,en,When Sheriff Roy Pulsipher finds himself in th...,4442.620,2022-11-15,R.I.P.D. 2: Rise of the Damned,6.8,175,"[sheriff, based on comic, american west, 1870s...",2022,"[Jeffrey Donovan, Penelope Mitchell, Richard B...","[{'crew_id': 22141, 'name': 'Ogden Gavanski', ..."
2,829799,en,Renegade bounty hunter Ryan Swan must carve hi...,3475.823,2022-11-11,Paradise City,6.3,28,[],2022,"[John Travolta, Bruce Willis, Blake Jenner]","[{'crew_id': 26713, 'name': 'Chuck Russell', '..."
3,872177,en,"Set in San Tiburon, the world's most dangerous...",3197.165,2022-04-29,Corrective Measures,5.0,31,"[based on comic, lockdown, prison measures]",2022,"[Bruce Willis, Hayley Sales, Michael Rooker]","[{'crew_id': 131416, 'name': ""Sean Patrick O'R..."
4,988233,en,Following a mysterious disappearance on a jump...,3648.142,2022-11-01,Hex,4.3,6,"[parachuting, paranormal, skydiving]",2022,"[Kayla Adams, Matthew Holcomb, Bryan David Rob...","[{'crew_id': 55873, 'name': 'Hans Rodionoff', ..."


In [565]:
def get_director(x):
    for i in ast.literal_eval(x):
        if i['department'] == 'Directing':
            return i['name']
    return np.nan

In [566]:
md['crew'] = md['crew'].apply(get_director)

In [567]:
md = md.rename(columns={'crew':'director'})

In [568]:
md['cast'] = md['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [569]:
md['director'] = md['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")).replace("-", ""))
md['director'] = md['director'].apply(lambda x: [x,x, x])

In [570]:
gen_md.columns

Index(['id', 'original_language', 'overview', 'popularity', 'release_date',
       'title', 'vote_average', 'vote_count', 'year', 'genre'],
      dtype='object')

In [571]:
md.head()

,id,original_language,overview,popularity,release_date,title,vote_average,vote_count,genres,year,cast,director
0,436270,en,"Nearly 5,000 years after he was bestowed with ...",11801.698,2022-10-19,Black Adam,7.3,2197,"[lightning, anti hero, superhero, based on com...",2022,"[dwaynejohnson, aldishodge, noahcentineo]","[jaumecolletserra, jaumecolletserra, jaumecoll..."
1,1013860,en,When Sheriff Roy Pulsipher finds himself in th...,4442.620,2022-11-15,R.I.P.D. 2: Rise of the Damned,6.8,175,"[sheriff, based on comic, american west, 1870s...",2022,"[jeffreydonovan, penelopemitchell, richardbrake]","[paulleyden, paulleyden, paulleyden]"
2,829799,en,Renegade bounty hunter Ryan Swan must carve hi...,3475.823,2022-11-11,Paradise City,6.3,28,[],2022,"[johntravolta, brucewillis, blakejenner]","[chuckrussell, chuckrussell, chuckrussell]"
3,872177,en,"Set in San Tiburon, the world's most dangerous...",3197.165,2022-04-29,Corrective Measures,5.0,31,"[based on comic, lockdown, prison measures]",2022,"[brucewillis, hayleysales, michaelrooker]","[seanpatricko'reilly, seanpatricko'reilly, sea..."
4,988233,en,Following a mysterious disappearance on a jump...,3648.142,2022-11-01,Hex,4.3,6,"[parachuting, paranormal, skydiving]",2022,"[kaylaadams, matthewholcomb, bryandavidroberts]","[chrisjohnston, chrisjohnston, chrisjohnston]"


In [572]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [573]:
# s = s.value_counts()
for i in range(200):
    for j in range(1):
        print(s.index[i+j])
        j = j + 1
    i = i + 20

0
0
0
0
0
0
0
0
1
1
1
1
1
3
3
3
4
4
4
5
5
5
5
5
5
5
5
5
5
5
5
5
6
6
8
8
8
8
8
9
9
9
9
9
9
9
11
13
13
13
13
13
13
13
13
15
15
15
15
15
15
15
15
15
15
15
15
15
15
16
17
18
18
18
18
18
18
19
19
19
19
19
20
20
21
21
21
21
21
21
22
22
22
22
23
24
24
24
24
24
24
24
24
24
24
24
24
24
25
25
25
29
29
29
30
30
30
30
31
32
35
35
35
35
35
35
35
35
35
36
36
36
36
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
38
38
38
38
38
38
38
38
39
39
39
39
39
39
39
39
39
39
40
40
40
40
40
41
43
43
43
43
43
43
43
43
43
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44


In [574]:
s = s[s > 1]

TypeError: '>' not supported between instances of 'str' and 'int'

In [ ]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

In [ ]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [ ]:
pkl.dump(md, open('allMovies.pkl', 'wb'))

In [ ]:
smd = md

In [ ]:
smd['genres'] = smd['genres'].apply(filter_keywords)
smd['genres'] = smd['genres'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['genres'] = smd['genres'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
smd['soup'] = smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [ ]:
smd.head(n = 5)

In [ ]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [ ]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [ ]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    movies = smd.iloc[movie_indices][['id', 'title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qfd = movies[
        (movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qfd['vote_count'] = qfd['vote_count'].astype('int')
    qfd['vote_average'] = qfd['vote_average'].astype('int')
    qfd['wr'] = qfd.apply(weighted_rating, axis=1)
    qfd = qfd.sort_values('wr', ascending=False).head(250)
    return qfd.drop(columns=['vote_count', 'vote_average'])

In [ ]:
improved_recommendations('The Dark Knight')

In [ ]:
qualified.iloc[0]['title']

In [ ]:
import pickle as pkl

In [ ]:
pkl.dump(qualified, open('topMovies.pkl', 'wb'))
pkl.dump(smd, open('allModifiedMovies.pkl', 'wb'))

In [ ]:
pkl.dump(cast_df, open('allCast.pkl', 'wb'))
pkl.dump(crew_df, open('allCrew.pkl', 'wb'))

In [ ]:
# smd = smd.drop(columns=['genres', 'cast', 'director'])

In [ ]:
smd.head(n=2)

In [ ]:
# pkl.dump(smd, open('allModifiedMovies.pkl', 'wb'))

In [ ]:
# improved_recommendations('The Dark Knight')

In [ ]:
# md.head(n = 3)

In [ ]:
# qualified.head(n = 2)

In [ ]:
# smd.head(n = 2)

In [ ]:
# gen_md.head(n = 2)

In [ ]:
# build_chart('romance')

In [ ]:
val = md[md['title'] == 'The Dark Knight']
val

In [ ]:
improved_recommendations('Toy Story')

,id,title,year,wr
860,68718,Django Unchained,2012,7.800399
469,10193,Toy Story 3,2010,6.834008
294,863,Toy Story 2,1999,6.823853
531,301528,Toy Story 4,2019,6.765231
674,920,Cars,2006,6.000000
327,49013,Cars 2,2011,6.000000
1209,50544,Friends with Benefits,2011,6.000000
741,49444,Kung Fu Panda 2,2011,6.000000
592,38365,Grown Ups,2010,6.000000
428,72105,Ted,2012,6.000000
